In [34]:
import re
import pickle
import pandas as pd
from sklearn.model_selection import train_test_split
from gensim.parsing.preprocessing import remove_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, classification_report
from xgboost import XGBClassifier

In [23]:
data = pd.read_csv("C:/Programs/Emotion Text Classification/text.csv")

print(data.shape)

(416809, 3)


In [24]:
print(data['text'][204])

filtered_text = remove_stopwords(data['text'][204])
print(filtered_text)

im becoming cynical in my old age my birthdays in two days and im feeling aggravated by circumstances
im cynical old age birthdays days im feeling aggravated circumstances


In [25]:
def remove_punctuation(sentence):
    pure_text = re.sub(r'(\s)?@\w+', r'\1', sentence)
    pure_text = re.sub(r'[^\w\s]', '', pure_text)
    pure_text = re.sub(r'\b.*com\b', '', pure_text)
    pure_text = re.sub(r'\bhttp\w+', '', pure_text)
    pure_text = pure_text.replace("  ", " ")
    pure_text = pure_text.lower()
    
    return pure_text

In [26]:

def preprocess(text):    
    text = remove_punctuation(text)
    
    text = remove_stopwords(text)
    
    return text


In [27]:
print(data['text'][200])
print(preprocess(data['text'][200]))

i m writing about myself now but when i look at them here i feel wonderfully privileged to have known so many like and unlike myself in such extreme situations
m writing look feel wonderfully privileged known like unlike extreme situations


In [28]:
data = data[['text', 'label']]

In [29]:
data['preprocessed_text'] = data['text'].apply(lambda a: preprocess(a))

In [30]:
data.head()

,text,label,preprocessed_text
0,i just feel really helpless and heavy hearted,4,feel helpless heavy hearted
1,ive enjoyed being able to slouch about relax a...,0,ive enjoyed able slouch relax unwind frankly n...
2,i gave up my internship with the dmrg and am f...,4,gave internship dmrg feeling distraught
3,i dont know i feel so lost,0,dont know feel lost
4,i am a kindergarten teacher and i am thoroughl...,4,kindergarten teacher thoroughly weary job havi...


In [31]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(data['preprocessed_text'])

In [32]:
y = data['label']

X_train, X_test, y_train, y_test = train_test_split(
     X, y, test_size=0.25, random_state=42)

In [33]:
y_train

319162    0
160826    1
280557    0
332236    0
206056    1
         ..
259178    2
365838    1
131932    3
146867    1
121958    4
Name: label, Length: 312606, dtype: int64

In [38]:
#xgb = XGBClassifier()
#xgb.fit(X_train, y_train)

#pickle.dump(xgb, open('xgb_model.pkl', 'wb'))

xgb = pickle.load(open('xgb_model.pkl', 'rb'))

xgb_pred = xgb.predict(X_test)
accuracy = accuracy_score(y_test, xgb_pred)

print(f"Accuracy is {accuracy}")

class_report = classification_report(y_test, xgb_pred)
print(class_report)

Accuracy is 0.8899647802846368
              precision    recall  f1-score   support

           0       0.94      0.92      0.93     30217
           1       0.92      0.89      0.90     35235
           2       0.74      0.91      0.82      8608
           3       0.92      0.88      0.90     14349
           4       0.89      0.81      0.85     11996
           5       0.68      0.90      0.78      3798

    accuracy                           0.89    104203
   macro avg       0.85      0.89      0.86    104203
weighted avg       0.90      0.89      0.89    104203



In [36]:
emotions = {0: "sadness", 1: "joy", 2: "love", 3: "anger", 4: "fear", 5: "surprise"}
# sadness (0), joy (1), love (2), anger (3), fear (4), and surprise (5)

In [41]:
new_sentence =[ "Congrats"]
vectorized = vectorizer.transform(new_sentence)

output = xgb.predict(vectorized)
emotions[output[0]]

'joy'